# XmR Chart Example

This notebook demonstrates the usage of the spc_plotly package with sample vehicle count data.

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from spc_plotly import xmr

%reload_ext autoreload
%autoreload 2

# Set random seed for reproducibility
np.random.seed(475)

## Make data

In [27]:
# Generate 100 days of data
start_date = datetime(2024, 1, 1)
dates = [start_date + timedelta(days=x) for x in range(100)]

# Generate vehicle counts with some patterns:
# - Base level around 1000 vehicles
# - Gradual upward trend
# - Some random variation
# - A few anomalies

base_counts = 1000
trend = np.linspace(0, 200, 100)  # Gradual increase over time
variation = np.random.normal(0, 50, 100)  # Random daily variation
weekly_pattern = np.tile([50, 10, 50, -20, -30, -80, -100], 15)[:100]  # Weekly patterns

# Combine components
vehicle_counts = base_counts + trend + variation + weekly_pattern

# Add some anomalies
vehicle_counts[25] += 300  # Special event
vehicle_counts[50:55] += 200  # Construction period
vehicle_counts[80] -= 400  # Road closure

# Create DataFrame
data = pd.DataFrame({
    'date': dates,
    'vehicle_counts': vehicle_counts.astype(int)
})

# Display first few rows
data.head()

,date,vehicle_counts
0,2024-01-01,942
1,2024-01-02,1013
2,2024-01-03,1073
3,2024-01-04,983
4,2024-01-05,1011


## Date example

In [28]:
data['date'] = pd.to_datetime(data['date'], utc=True)

In [29]:
# Create XmR chart
xmr_chart = xmr.XmR(
    data=data,
    x_type='date_time',
    x_ser_name='date',
    y_ser_name='vehicle_counts',
    date_part_resolution='day',
    title='Daily Vehicle Counts - XmR Chart',
    # period_breaks = ['2024-01-21', '2024-03-01']
)

# Display the chart
xmr_chart.xmr_chart

In [30]:
# Check the calculated limits and signals
print("Moving Range Limits:")
print(xmr_chart.mR_limit_values)
print("\nNatural Process Limits:")
print(xmr_chart.npl_limit_values)
print("\nDetected Signals:")
print(xmr_chart.signals)
# xmr_chart.y_xmr_func

Moving Range Limits:
{'mR_xmr_func': np.float64(100.18181818181819), 'mR_upper_limit': np.float64(327.39418181818183), 'xmr_func': 'mean'}

Natural Process Limits:
{'y_xmr_func': np.float64(1089.74), 'npl_upper_limit': np.float64(1356.2236363636364), 'npl_lower_limit': np.float64(823.2563636363636), 'xmr_func': 'mean'}

Detected Signals:
{'anomalies': [(np.datetime64('2024-01-14T00:00:00.000000000'), np.int64(820), 'Low'), (np.datetime64('2024-02-21T00:00:00.000000000'), np.int64(1367), 'High'), (np.datetime64('2024-03-21T00:00:00.000000000'), np.int64(703), 'Low')], 'long_runs': [[(Timestamp('2024-01-01 00:00:00'), np.int64(942), 'Low'), (Timestamp('2024-01-02 00:00:00'), np.int64(1013), 'Low'), (Timestamp('2024-01-03 00:00:00'), np.int64(1073), 'Low'), (Timestamp('2024-01-04 00:00:00'), np.int64(983), 'Low'), (Timestamp('2024-01-05 00:00:00'), np.int64(1011), 'Low'), (Timestamp('2024-01-06 00:00:00'), np.int64(883), 'Low'), (Timestamp('2024-01-07 00:00:00'), np.int64(859), 'Low'), (T

In [41]:
# Numeric example
numeric_data = pd.DataFrame({
    'position': range(1, 101),
    'measurement': np.random.normal(100, 10, 100)
})

xmr_numeric = xmr.XmR(
    data=numeric_data,
    x_ser_name='position',
    y_ser_name='measurement',
    x_type='numeric',
    chart_height=800,
    period_breaks=[40, 45, 50],
    period_end=70
)

xmr_numeric.xmr_chart

In [37]:
# Categorical example
categories = ['Station_' + chr(ord('a') + i) for i in range(100)]
categorical_data = pd.DataFrame({
    'station': categories,
    'quality_score': np.random.normal(85, 5, 100)
}).sort_values('station')

xmr_categorical = xmr.XmR(
    data=categorical_data,
    x_ser_name='station',
    y_ser_name='quality_score',
    x_type='categorical',
    period_breaks = ["Station_f"],
    period_end = ["Station_z"]
    
)

xmr_categorical.xmr_chart

In [33]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

# Generate 100 days of data
start_date = datetime(2024, 1, 1)
dates = [start_date + timedelta(days=x) for x in range(100)]

# Generate vehicle counts with controlled patterns
base_counts = 1000
trend = np.linspace(0, 150, 100)  # Gradual increase over time

# Create long runs
long_run_length = 20
long_run_value = 1100  # Value for long runs
short_run_value = 1050  # Value for short runs

# Initialize vehicle counts with the base level
vehicle_counts = np.full(100, base_counts)

# Introduce long runs
for i in range(0, 100, long_run_length):
    if i + long_run_length <= 100:
        vehicle_counts[i:i + long_run_length] = long_run_value + trend[i:i + long_run_length]

# Introduce short runs
short_run_indices = [10, 40, 70]  # Starting points for short runs
for idx in short_run_indices:
    if idx + 5 <= 100:  # Ensure we don't go out of bounds
        vehicle_counts[idx:idx + 5] = short_run_value + trend[idx:idx + 5]

# Add some anomalies
vehicle_counts[25] += 300  # Special event
vehicle_counts[50:55] += 200  # Construction period
vehicle_counts[80] -= 400  # Road closure

# Create DataFrame with an additional column for numeric values from 1 to 100
data_controlled = pd.DataFrame({
    'date': dates,
    'vehicle_counts': vehicle_counts.astype(int),
    'numeric_values': range(1, 101)  # New column with numeric values from 1 to 100
})

# Display first few rows
data_controlled.head()

,date,vehicle_counts,numeric_values
0,2024-01-01,1100,1
1,2024-01-02,1101,2
2,2024-01-03,1103,3
3,2024-01-04,1104,4
4,2024-01-05,1106,5


In [45]:
# data_controlled[['date', 'vehicle_counts']].to_clipboard()

In [42]:
# Create XmR chart
xmr_chart = xmr.XmR(
    data=data_controlled,
    x_type='date_time',
    x_ser_name='date',
    y_ser_name='vehicle_counts',
    date_part_resolution='day',
    title='Daily Vehicle Counts - XmR Chart',
    period_breaks = ['2024-01-21', '2024-03-01'],
    # period_end = '2024-01-03',
    y_axis_dtick = 100
    
)

# Display the chart

xmr_chart.xmr_chart

In [44]:
xmr_chart.npl_limit_values

{'y_xmr_func': [np.float64(1102.857142857143),
  np.float64(1185.7073170731708),
  np.float64(1203.7)],
 'npl_upper_limit': [np.float64(1119.615142857143),
  np.float64(1262.1823170731707),
  np.float64(1268.6312820512821)],
 'npl_lower_limit': [np.float64(1086.0991428571429),
  np.float64(1109.2323170731709),
  np.float64(1138.768717948718)],
 'xmr_func': 'mean'}